1.using question_user_page as benchmark, split quetsion_title or question_body into train_test, we believe different website have different type questions, so we can make evaluate and predict model

2.Using unsupervise learning to cluster question into differnt type, culster is depending on data preprocessing granularity. smaller grandularity, more spase cluster

3.After i embedding these sentence, you can use KNN SVM to do unsuperviese cluster

4.Using categore to cluster by CNN(n-gram / Glove / miniGPT）

5.Generage numerical value by former data and compart to anser_well_written

6.

In [2]:
import pandas as pd

In [3]:
data_1 = pd.read_csv(r"D:\Downloads\google-quest-challenge\train.csv")

In [4]:
data_1.shape

(6079, 41)

In [6]:
data_1.columns

Index(['qa_id', 'question_title', 'question_body', 'question_user_name',
       'question_user_page', 'answer', 'answer_user_name', 'answer_user_page',
       'url', 'category', 'host', 'question_asker_intent_understanding',
       'question_body_critical', 'question_conversational',
       'question_expect_short_answer', 'question_fact_seeking',
       'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written', 'answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfa

In [20]:
data_1.iloc[4773,9]

'CULTURE'

In [26]:
data_1.iloc[0:5,[5,9]]

,answer,category
0,"I just got extension tubes, so here's the skin...",LIFE_ARTS
1,It might be helpful to look into the definitio...,CULTURE
2,Do you even need grooves? We make several pro...,SCIENCE
3,"Sending an ""affidavit"" it is a dispute between...",CULTURE
4,Check out Image Trace in Adobe Illustrator. \n...,LIFE_ARTS


In [8]:
data_1.head(5)

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,ysap,https://photo.stackexchange.com/users/1024,"I just got extension tubes, so here's the skin...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000
1,1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,russellpierce,https://rpg.stackexchange.com/users/8774,It might be helpful to look into the definitio...,Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/w...,CULTURE,...,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889
2,2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Joe Baker,https://electronics.stackexchange.com/users/10157,Do you even need grooves? We make several pro...,Dwayne Reid,https://electronics.stackexchange.com/users/64754,http://electronics.stackexchange.com/questions...,SCIENCE,...,0.777778,0.777778,0.555556,1.000000,1.000000,0.666667,0.0,0.333333,1.000000,0.888889
3,3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...",Scimonster,https://judaism.stackexchange.com/users/5151,"Sending an ""affidavit"" it is a dispute between...",Y e z,https://judaism.stackexchange.com/users/4794,http://judaism.stackexchange.com/questions/551...,CULTURE,...,0.888889,0.833333,0.333333,0.833333,1.000000,0.800000,0.0,0.000000,1.000000,1.000000
4,5,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,leigero,https://graphicdesign.stackexchange.com/users/...,Check out Image Trace in Adobe Illustrator. \n...,q2ra,https://graphicdesign.stackexchange.com/users/...,http://graphicdesign.stackexchange.com/questio...,LIFE_ARTS,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,1.000000,1.000000


In [8]:
data_1.iloc[2000:2005,:]

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
2000,3193,Connect OpenLayers3 Client with QGIS Server,at the moment I am developing a map editor on ...,Alcaamado,https://gis.stackexchange.com/users/44972,Yes you can for example use the WMS-Layer from...,Thomas B,https://gis.stackexchange.com/users/45041,http://gis.stackexchange.com/questions/130200/...,TECHNOLOGY,...,0.555556,1.000000,0.666667,1.000000,1.000000,1.000000,0.666667,0.0,0.333333,0.888889
2001,3195,Examples of two different descriptions of a se...,I am teaching a course in enumerative combinat...,Ben Braun,https://mathoverflow.net/users/10602,Here is an example where the question of wheth...,Joel David Hamkins,https://mathoverflow.net/users/1946,http://mathoverflow.net/questions/52899,SCIENCE,...,0.888889,0.833333,0.444444,0.833333,0.833333,0.600000,0.000000,0.0,0.666667,0.888889
2002,3196,Rails not rendering a partial on webpage,total newbie trying to work through the rails ...,James Lin,https://stackoverflow.com/users/714411,You need the = sign:\n\n&lt;%= render :partial...,robotcookies,https://stackoverflow.com/users/941500,http://stackoverflow.com/questions/7935510/rai...,STACKOVERFLOW,...,0.555556,1.000000,0.666667,1.000000,1.000000,1.000000,1.000000,0.0,0.000000,0.888889
2003,3197,Google stopped indexing my site. Is there a su...,On my site is a page that hosts all my ad entr...,user1838185,https://webmasters.stackexchange.com/users/30221,"Without seeing your site, it sounds like you a...",JCL1178,https://webmasters.stackexchange.com/users/5757,http://webmasters.stackexchange.com/questions/...,TECHNOLOGY,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.866667,0.666667,0.0,1.000000,1.000000
2004,3198,What is a *slightly* less extreme equivalent t...,tl;dr: What is a less extreme (but still notic...,Mehrdad,https://programmers.stackexchange.com/users/11833,"Would ""proficient"" be useful, if not that, ""co...",Irwin,https://programmers.stackexchange.com/users/35979,http://programmers.stackexchange.com/questions...,TECHNOLOGY,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.933333,0.000000,0.0,0.333333,0.888889
